In [1]:
from pathlib import Path
import scanpy as sc
import re, sys
import pandas as pd

In [2]:
def parse_sam(file):
	result = set()
	with open(file, 'r') as inputFile:
		for line in inputFile.readlines():
			fields = line.split('\t')
			acc = fields[2]
			for item in fields[19:]:
				if item.find('CB:Z:') > -1:
					barcode = re.sub(r'CB:Z:', r'', item)
				if item.find('UB:Z:') > -1:
					umi = re.sub(r'UB:Z:', '', item)
			result.add(f'{acc}\t{barcode}\t{umi}')
	return result

def generate_table(data, target_set, target_name, suffix):
	filtered_data = []
	for item in data:
		if item.split('\t')[0] in target_set:
			filtered_data.append(item)

	with open('exp_aae_denv.txt', 'w') as outputFile:
		for item in filtered_data:
			outputFile.write(item)
	table = pd.read_table('exp_aae_denv.txt', header=None, delimiter='\t')
	table.columns = ['Accession', 'Barcode', 'UMI']
	table['Accession'] = table['Accession'].map(lambda x: target_name[x])
	table = table[table['Barcode'] != '-']
	table = table[table['UMI'] != '-']
	table = table.groupby(['Accession', 'Barcode'])['UMI'].count().reset_index()
	table['Barcode'] = table['Barcode'] + suffix
	Path('exp_aae_denv.txt').unlink()
	return table

def write2csv(adata, table, target_name, suffix):
	umap_df = pd.DataFrame(
	adata.obsm['X_umap'], 
	index=adata.obs_names,
	columns=['UMAP_1', 'UMAP_2']  # 列名可自定义
	)
	umap_df = pd.concat([adata.obs['cluster'], umap_df], axis=1).reset_index()
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2']
	umap_df.to_csv('temp.csv', header=True, index=False)
	umap_df = pd.read_csv('temp.csv', header=0)
	for name in target_name.values():
		temp_column = table[table['Accession'] == name][['Barcode', 'UMI']]
		umap_df = umap_df.merge(temp_column, how='left', on='Barcode').fillna(0)
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2'] + list(target_name.values())
	umap_df.to_csv(f'exp_{suffix}.csv', index=False)
	Path('temp.csv').unlink()
	print(umap_df)

In [3]:
data = parse_sam('exp_cxtri_bl.sam')

In [4]:
target_set = ['MW174761.1', 'ON949933.1']
target_name = {
	'MW174761.1': 'Totichi',
	'ON949933.1': 'HKIFV'
}
table = generate_table(data, target_set, target_name, '-bl')

In [5]:
adata = sc.read('../../../original_h5ad/cxtri_cxtri_bl_harmony.h5ad')
adata = adata[adata.obs['batch'] == 'cxtri-bl']
write2csv(adata, table, target_name, 'cxtri_bl')

                               Barcode    Cluster     UMAP_1     UMAP_2  \
0     AAGCTTGCG_ACACCAACG_AACAAGTGG-bl   Cardia-1  16.185768   0.118158   
1     AGGACTCAC_ACCAGGTCA_AACAAGTGG-bl     ISC/EB   5.975157  10.801889   
2     GTGCAAGGT_ACCTGCTAC_AACAAGTGG-bl         EC   8.091039   0.237152   
3     GTGTTAAGG_AGGCTGTTG_AACAAGTGG-bl     ISC/EB   7.091428  12.146221   
4     GTGCAAGGT_AGTGGTGGT_AACAAGTGG-bl     ISC/EB   5.483441  10.507774   
...                                ...        ...        ...        ...   
4530  GGCTTCTCA_GGCATGCAA_TTGTGTACG-bl       EE-3  13.357920   2.292581   
4531  TCGGACATG_TACTCACCA_TTGTGTACG-bl  EC-like-3   8.538496   3.405360   
4532  ACGAAGCTC_TGAGGCCTT_TTGTGTACG-bl       EE-1  11.035083   8.885902   
4533  TCACTGGAA_TGGACTTGT_TTGTGTACG-bl         VM  -0.153450   7.155228   
4534  GAACGCTAT_TGGTCAGTT_TTGTGTACG-bl   Cardia-2  14.670137   6.394145   

      Totichi  HKIFV  
0         0.0    4.0  
1         1.0    4.0  
2         1.0    6.0  
3      